# Anomaly Detection Using Keras
*** 

In [5]:
import keras  #[2]


### References
https://keras.io/examples/timeseries/timeseries_anomaly_detection/
https://techvidvan.com/tutorials/keras-installation/

https://www.kaggle.com/code/andreshg/timeseries-analysis-a-complete-guide
